In [ ]:
include("settings.jl")

## Plot Ephys Raw data

Plot things like bout-aligned spike rasters etc.

In [ ]:
bouts = convert(DataFrame, py"get_bouts('outbound', baseline=False, target='MOs')");

In [ ]:
py"""

def get_bouts_activity(name, bw=25):
    activity = pd.DataFrame(
        Probe.RecordingSite * Unit * FiringRate & f"name='{name}'"  & f"bin_width = {bw}"
    )
    
    return activity
"""


regions = Dict(
    "MOs" => ["MOs1", "MOs2/3", "MOs4", "MOs5", "MOs6a", "MOs6b"],
    "MRN" => ["CUN", "PPN"]
)

function add_activity_to_bouts!(bouts; target="MOs")
    @assert length(unique(bouts.name)) == 1 "bouts must be from the same recording"
    rec = bouts.name[1]
    recording_activity = convert(DataFrame, py"get_bouts_activity"(rec, bw=25))

    # keep units in the right regions
    recording_activity = recording_activity[recording_activity.brain_region .∈ regions[target], :]
    @info "got activity" size(bouts) size(recording_activity)

    # cut activity for each bout
    # activity = Dict(
    #     "unit_$(u)"=>[] for u in recording_activity.unit_id 
    # )
    
    # for u in units.unit_id
    #     unit_activity::Vector = recording_activity[recording_activity.unit_id .== u, :]

    #     for (i, bout) in enumerate(bouts)
    #         start_frame = bout.corrected_start_frame
    #         end_frame = bout.corrected_end_frame
    #         push!(activity["unit_$(u)"],unit_activity[start_frame:end_frame])
    #     end

    #     break
    # end

    # for (k, v) in activity
    #     bouts[k] = v
    # end
    
end
nothing

In [ ]:
recs = bouts.name |> unique

rec = recs[13]
rec_bouts = bouts[bouts.name .== rec, :]

# activity = convert(DataFrame, py"get_bouts_activity"(rec, bw=25)
# activity = py"get_bouts_activity"(rec, bw=25)

add_activity_to_bouts!(rec_bouts, target="MOs")
